# 긴 시퀀스 다루기
긴 시퀀스로 기본 RNN을 학습하려면 많은 타임 스텝에 걸쳐 실행해야 하므로 펼친 RNN이 매우 깊은 네트워크가 됨.  
보통의 심층 신경망처럼 
- 그레디언트 소실과 폭주 문제를 가질 수 있음. 학습하는 데 시간이 오래 걸리고 불안정할 수 있음.  
- 또한 RNN이 긴 시퀀스를 처리할 때 입력의 첫 부분을 조금씩 잊어버릴 것임.

---
## 불안정한 그레디언트 문제
이전에 심층 신경망에서 사용했던 기법들
- 좋은 가중치 초기화 및 활성 함수
- 빠른 옵티마이저
- 드롭아웃 등

**하지만 RNN에선 수렴하지 않은 활성 함수(ReLU같은)는 학습할 때 더 불안정하게 만든다고 함.** 이는 그레디언트 폭주를 일으킬 수 있다고 함.  
그래서 **하이퍼볼릭 탄젠트**같은 함수를 많이 사용한다고 함.  
> 그래도 그레디언트가 폭주하면 **그레디언트 클리핑** 활용

**배치 정규화도 RNN에 효율적으로 사용할 수는 없다고 함**. 타임 스텝 사이에는 사용할 수 없고 층 사이에만 가능함.  
대신 **층 정규화 (layer normalization)** 이 잘 맞다고 함.
> 배치 정규화와 비슷하지만 배치 차원에 대해 정규화하는 대신에 **특성 차원에 대해 정규화함.**  
샘플에 독립적으로 타임 스텝마다 동적으로 필요한 통계를 계산하고 이는 학습과 테스트에서 동일한 방식으로 작동한다는 것을 의미.  

In [1]:
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [2]:
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)
    
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

**층 정규화를 적용한 RNN cell 예시**

In [9]:
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True, input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

---
## LSTM

In [10]:
lstm_model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [11]:
import numpy as np

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5*np.sin((time - offset1)*(freq1*10 + 10))
    series += 0.2*np.sin((time - offset2)*(freq2*20 + 20))
    series += (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [12]:
n_steps = 50

series = generate_time_series(10000, n_steps+10)
X_train, Y_train = series[: 7000, :n_steps], series[: 7000, -10:, 0]
X_valid, Y_valid = series[7000: 9000, :n_steps], series[7000: 9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10+1):
    Y[:, :, step_ahead-1] = series[:, step_ahead:step_ahead+n_steps, 0]

Y_train = Y[: 7000]
Y_valid = Y[7000: 9000]
Y_test = Y[9000: ]

In [13]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
lstm_model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])

In [14]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), batch_size=256)

Epoch 1/20
28/28 [==============================] - 9s 272ms/step - loss: 0.5698 - last_time_step_mse: 0.5685 - val_loss: 0.2936 - val_last_time_step_mse: 0.2927
Epoch 2/20
28/28 [==============================] - 7s 254ms/step - loss: 0.2567 - last_time_step_mse: 0.2524 - val_loss: 0.1962 - val_last_time_step_mse: 0.1841
Epoch 3/20
28/28 [==============================] - 7s 260ms/step - loss: 0.1878 - last_time_step_mse: 0.1744 - val_loss: 0.1753 - val_last_time_step_mse: 0.1623
Epoch 4/20
28/28 [==============================] - 7s 255ms/step - loss: 0.1727 - last_time_step_mse: 0.1597 - val_loss: 0.1694 - val_last_time_step_mse: 0.1567
Epoch 5/20
28/28 [==============================] - 7s 255ms/step - loss: 0.1673 - last_time_step_mse: 0.1551 - val_loss: 0.1677 - val_last_time_step_mse: 0.1553
Epoch 6/20
28/28 [==============================] - 7s 258ms/step - loss: 0.1653 - last_time_step_mse: 0.1527 - val_loss: 0.1634 - val_last_time_step_mse: 0.1514
Epoch 7/20
28/28 [==========

In [15]:
lstm_model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), batch_size=256)

Epoch 1/20
28/28 [==============================] - 3s 29ms/step - loss: 0.2259 - last_time_step_mse: 0.2247 - val_loss: 0.2034 - val_last_time_step_mse: 0.1989
Epoch 2/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1921 - last_time_step_mse: 0.1847 - val_loss: 0.1720 - val_last_time_step_mse: 0.1629
Epoch 3/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1700 - last_time_step_mse: 0.1609 - val_loss: 0.1636 - val_last_time_step_mse: 0.1542
Epoch 4/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1623 - last_time_step_mse: 0.1521 - val_loss: 0.1588 - val_last_time_step_mse: 0.1498
Epoch 5/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1572 - last_time_step_mse: 0.1480 - val_loss: 0.1562 - val_last_time_step_mse: 0.1480
Epoch 6/20
28/28 [==============================] - 0s 9ms/step - loss: 0.1551 - last_time_step_mse: 0.1467 - val_loss: 0.1542 - val_last_time_step_mse: 0.1457
Epoch 7/20
28/28 [=================

>오 LSTM이 학습 속도도 훨씬 빠르고 성능도 좋음

---
## GRU
**게이트 순환 유닛 (gated recurrent unit)** 은 LSTM의 변종으로 더 간소화되고 유사하게 작동함.

In [16]:
gru_model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [17]:
gru_model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
gru_model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid), batch_size=256)

Epoch 1/20
28/28 [==============================] - 3s 26ms/step - loss: 0.2283 - last_time_step_mse: 0.2285 - val_loss: 0.2066 - val_last_time_step_mse: 0.2044
Epoch 2/20
28/28 [==============================] - 0s 9ms/step - loss: 0.1976 - last_time_step_mse: 0.1931 - val_loss: 0.1768 - val_last_time_step_mse: 0.1744
Epoch 3/20
28/28 [==============================] - 0s 9ms/step - loss: 0.1717 - last_time_step_mse: 0.1679 - val_loss: 0.1608 - val_last_time_step_mse: 0.1564
Epoch 4/20
28/28 [==============================] - 0s 8ms/step - loss: 0.1581 - last_time_step_mse: 0.1508 - val_loss: 0.1544 - val_last_time_step_mse: 0.1483
Epoch 5/20
28/28 [==============================] - 0s 8ms/step - loss: 0.1544 - last_time_step_mse: 0.1463 - val_loss: 0.1534 - val_last_time_step_mse: 0.1475
Epoch 6/20
28/28 [==============================] - 0s 9ms/step - loss: 0.1531 - last_time_step_mse: 0.1450 - val_loss: 0.1528 - val_last_time_step_mse: 0.1469
Epoch 7/20
28/28 [=====================

>LSTM 보다 훨씬 학습이 빠르게 됨.  
성능 자체는 LSTM이 더 좋은듯?